# Задача

Делаем анализ, который обсуждали.

*Исходник данных*: продажи за 2 года.

1. На 2016.12.31 получаем инфу по кол-ву заказов на каждого юзера. Подсчитываем сколько пользователей попало в группу с 1 заказом, 2, 3 и т.д.
2. За январь 2017 смотрим, какое кол-во юзеров из п.1 купило в каждой группе и на какую сумму за месяц.(скрипты вам должны помочь)
3. Подсчитываем вероятность перехода в покупающего по каждой группе и ср траты за январь в группе (как на занятии).
4. В идеале сделать тоже самое на февраль, март, и до конца 17 года (чтобы оценить сезонность и рост трат за месяц)

# Подготовка

## Подключение библиотек

In [1]:
import pymysql.cursors
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import getpass

## Подключение к базе данных

In [2]:
# password_mysql = getpass.getpass(prompt="Введите пароль от MySQL-сервера: ")
password_mysql = "pass123"
mydb = pymysql.connect(host="127.0.0.1",
                             user='root',
                             password=password_mysql, 
                             db='db_analyt_loc',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
 
print ("connect successful!!")

connect successful!!


In [5]:
def show_tables():
    return pd.read_sql("""SHOW TABLES FROM db_analyt_loc;""", con=mydb)
show_tables()

,Tables_in_db_analyt_loc
0,orders_20190822
1,orders_all
2,time_user_b
3,time_user_ym_s
4,users_for_merg


In [7]:
mydb.cursor().execute(
    """
    DROP TABLE IF EXISTS one_month_table;
    """)
mydb.commit()

0

In [8]:
mydb.cursor().execute(
    """
    CREATE TABLE one_month_table (
    SELECT 
        user_id,
        COUNT(id_o) AS n,
        MONTH(o_date) AS date_m,
        YEAR(o_date) AS date_y
    FROM orders_20190822 o
    WHERE price BETWEEN 0 AND 150000
    GROUP BY user_id, date_m, date_y);
    """)
mydb.commit()

In [38]:
df_1 = pd.read_sql(f"SELECT * FROM one_month_table;", con=mydb)
df_1.head()

,user_id,n,date_m,date_y
0,337544,1,1,2016
1,171642,4,1,2016
2,260596,1,1,2016
3,1105609,1,1,2016
4,982696,1,1,2016


In [37]:
df_1.loc[df_1["n"] < 5, 'buyer_group'] = 1
df_1.loc[(df_1["n"] >= 5) & ((df_2["n"] < 10)), 'buyer_group'] = 2
df_1.loc[(df_1["n"] >= 10) & ((df_2["n"] < 20)), 'buyer_group'] = 3
df_1.loc[df_1["n"] >= 20, 'buyer_group'] = 4

print(df_1.buyer_group.value_counts())

df_1.head()

1.0    1507763
2.0       1434
4.0        819
3.0        178
Name: buyer_group, dtype: int64


,user_id,n,date_m,date_y,buyer_group
0,337544,1,1,2016,1.0
1,171642,4,1,2016,1.0
2,260596,1,1,2016,1.0
3,1105609,1,1,2016,1.0
4,982696,1,1,2016,1.0


### 1. На 2016.12.31 получаем инфу по кол-ву заказов на каждого юзера. Подсчитываем сколько пользователей попало в группу с 1 заказом, 2, 3 и т.д.

In [50]:
df_1701 = df_1[df_1['date_y'] == 2016].groupby('user_id').aggregate(sum)
df_1701['y'] = 1701
df_1701.head()
df_1701 = df_1701.drop(['date_m', 'date_y'], axis=1)

1      334087
2       49782
3       20067
4       11272
5        7204
        ...  
132         1
647         1
136         1
285         1
510         1
Name: n, Length: 156, dtype: int64

In [52]:
df_1701.loc[df_1701["n"] < 5, 'buyer_group'] = 1
df_1701.loc[(df_1701["n"] >= 5) & ((df_2["n"] < 10)), 'buyer_group'] = 2
df_1701.loc[(df_1701["n"] >= 10) & ((df_2["n"] < 20)), 'buyer_group'] = 3
df_1701.loc[df_1701["n"] >= 20, 'buyer_group'] = 4

df_1701.head()

,n,y,buyer_group
user_id,,,
0,1,1701,1.0
1,1,1701,1.0
76,1,1701,1.0
90,1,1701,1.0
91,3,1701,1.0


In [59]:
df_1701.shape

(445089, 3)

In [17]:
df_1702 = df_1[df_1['date_y'] in ].groupby('user_id').aggregate(sum)
df_1702['y'] = 1702
df_1702 = df_1702.drop(['date_m', 'date_y'], axis=1)
df_1702.head()

date_m,1,2,3,4,5,6,7,8,9,10,11,12
n,,,,,,,,,,,,
1,28981.0,28082.0,34952.0,36778.0,30217.0,31996.0,31653.0,36573.0,40831.0,53736.0,72254.0,85585.0
2,4454.0,4236.0,5559.0,6598.0,5416.0,6031.0,5632.0,6146.0,6592.0,8294.0,10279.0,10646.0
3,1073.0,1020.0,1583.0,2080.0,1586.0,1808.0,1678.0,1792.0,1936.0,2394.0,2856.0,2772.0
4,358.0,321.0,636.0,806.0,620.0,743.0,650.0,709.0,730.0,887.0,1043.0,898.0
5,171.0,132.0,232.0,408.0,252.0,329.0,301.0,347.0,334.0,368.0,475.0,367.0
...,...,...,...,...,...,...,...,...,...,...,...,...
214,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


### 2. За январь 2017 смотрим, какое кол-во юзеров из п.1 купило в каждой группе и на какую сумму за месяц.(скрипты вам должны помочь)

### 3. Подсчитываем вероятность перехода в покупающего по каждой группе и ср траты за январь в группе (как на занятии).

### 4. В идеале сделать тоже самое на февраль, март, и до конца 17 года (чтобы оценить сезонность и рост трат за месяц)

# Закрытие сессии

In [60]:
mydb.close()